In [26]:
%matplotlib inline

In [27]:
#!/usr/bin/python

import cv2
import numpy as np
import random

In [28]:

import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from skimage import data
import sknw
import networkx as nx

In [32]:

# Check if a point is inside a rectangle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Draw a point
def draw_point(img, p, color ) :
    # cv2.circle( img, p, 2, color, cv2.cv.CV_FILLED, cv2.LINE_AA, 0 )
    cv2.circle( img, p, 2, color, cv2.FILLED, cv2.LINE_AA, 0 )

# Draw delaunay triangles
def draw_delaunay(img, subdiv, delaunay_color ) :

    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList :

        pt1 = (    int( t[0] )   ,   int( t[1] )  )
        pt2 = (    int( t[2] )   ,   int( t[3] )  )
        pt3 = (    int( t[4] )   ,   int( t[5] )  )

        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :

            cv2.line(img, pt1, pt2, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt2, pt3, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt3, pt1, delaunay_color, 1, cv2.LINE_AA, 0)

# Draw voronoi diagram
def draw_voronoi(img, subdiv) :

    ( facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)) :
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)

        ifacet = np.array(ifacet_arr, np.int)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        cv2.fillConvexPoly(img, ifacet, color, cv2.LINE_AA, 0);
        ifacets = np.array([ifacet])
        cv2.polylines(img, ifacets, True, (0, 0, 0), 1, cv2.LINE_AA, 0)
        # cv2.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), cv2.cv.CV_FILLED, cv2.LINE_AA, 0)
        cv2.circle(img, (int(centers[i][0]), int(centers[i][1])), 3, (0, 0, 0), cv2.FILLED, cv2.LINE_AA, 0)

def open_image(path):
    try: 
        image = cv2.imread(path, 0) # Reading an image in grayscale mode
        print("found image at : " , path)
    except:
        print("couldn't find image at : ", path)
    return(image)



In [33]:
#skeleton to graph definition

def skeleton_to_graph(path):
    try: img = cv2.imread(cv2.samples.findFile(path),0) # get img
    except : print("can't find that image at", path)
    img = (255-img) 
    img = img > 127 #make bool 
    ske = skeletonize(~img).astype(np.uint16) # use sknw skeletonize #----------------------------this is including skeletonize!!!! 
    graph = sknw.build_sknw(ske) # build graph from skeleton
    print("\nskeleton converted to graph\n")
    return graph


In [31]:


if __name__ == '__main__':

    # Define window names
    win_delaunay = "Delaunay Triangulation"
    win_voronoi = "Voronoi Diagram"

    # Turn on animation while drawing triangles
    animate = True

    # Define colors for drawing.
    delaunay_color = (255,255,255)
    points_color = (0, 0, 255)

    # Read in the image. -----------------------------
    path = r'../Lake_Murray_Map_Skeletons/SE_corner_skeleton.png' 
    img = open_image(path)
    

    # Keep a copy around
    img_orig = img.copy()

    # Rectangle to be used with Subdiv2D
    size = img.shape
    rect = (0, 0, size[1], size[0])

    # Create an instance of Subdiv2D
    subdiv = cv2.Subdiv2D(rect)

    #------------------------------------get graph

    # path = r'../Lake_Murray_Map_Skeletons/SE_corner_medial_axis.png' 
    graph = skeleton_to_graph(path)
    image = open_image(path) #start by opening the image, choose image in the function 


    #------------------------------------get points
    # Create an array of points.
    points = []

    for (s,e) in graph.edges():
        ps = graph[s][e]['pts']
        
        #print(len(ps))
        if len(ps)%2 == 0:
            midpoint = [ ps[int(len(ps)/2),1] , ps[int(len(ps)/2),0] ]
            #print(int(len(ps)/2))
        else:
            midpoint = [ ps[int((len(ps)+1) /2),1] , ps[int((len(ps)+1) /2),0] ]
            #print(int((len(ps)+1) /2))
        # print(len(ps),midpoint)
        # plt.plot(midpoint[0], midpoint[1], 'b.')
        points.append( ( int(midpoint[0]) , int(midpoint[1]) ) )


    # Read in the points from a text file
    # with open("points.txt") as file :
    #     for line in file :
    #         x, y = line.split()
    #         points.append((int(x), int(y)))



    # Insert points into subdiv
    for p in points :
        subdiv.insert(p)

        # Show animation
        if animate :
            img_copy = img_orig.copy()
            # Draw delaunay triangles
            draw_delaunay( img_copy, subdiv, (255, 255, 255) );
            cv2.imshow(win_delaunay, img_copy)
            cv2.waitKey(100)

    # Draw delaunay triangles
    draw_delaunay( img, subdiv, (255, 255, 255) );

    # Draw points
    for p in points :
        draw_point(img, p, (0,0,255))

    # Allocate space for Voronoi Diagram
    img_voronoi = np.zeros(img.shape, dtype = img.dtype)

    # Draw Voronoi diagram
    draw_voronoi(img_voronoi,subdiv)

    # Show results
    cv2.imshow(win_delaunay,img)
    cv2.imshow(win_voronoi,img_voronoi)
    cv2.waitKey(0)


found image at :  ../Lake_Murray_Map_Skeletons/SE_corner_skeleton.png

skeleton converted to graph

found image at :  ../Lake_Murray_Map_Skeletons/SE_corner_skeleton.png
<ipython-input-29-388f2f5c2ee9>:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ifacet = np.array(ifacet_arr, np.int)


error: OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
